In [2]:
from n_body_system import NBodySystem
import torch
import numpy as np

# Create N-body system

In [9]:
dim = 2        # dimension
num_bodies = 5 # number of bodies

force_1_over_r = lambda r, G, m1, m2: G * m1 * m2 / (r + 1e-8)
system = NBodySystem(N=num_bodies, dimension=dim, G=1, R=1, force_law=force_1_over_r, seed=42)

# Run dynamics 

In [10]:
dt = 0.005 
num_steps = 5    # number of time steps

# Preallocate arrays
nodes_t = np.zeros((num_steps, num_bodies, 2*dim+1))
vel_updated_true = np.zeros((num_steps, num_bodies, dim))

# Run simulation
for t in range(num_steps):
    nodes_t[t, :, 0:dim] = system.positions
    nodes_t[t, :, dim:2*dim] = system.velocities
    nodes_t[t, :, -1] = system.masses.flatten()

    system.step(dt)  # advance velocities

    vel_updated_true[t, :, :] = system.velocities.copy()

print("nodes_t shape:", nodes_t.shape)              # (num_steps, num_bodies, 2*dim+1)
print("vel_updated_true shape:", vel_updated_true.shape)  # (num_steps, num_bodies, dim)

nodes_t shape: (5, 5, 5)
vel_updated_true shape: (5, 5, 2)


# Create sender and receiver indices

In [11]:
# Create all pairs (i,j) where i != j
senders, receivers = np.meshgrid(np.arange(num_bodies), np.arange(num_bodies))
senders = senders.flatten()
receivers = receivers.flatten()

# Remove self-loops
mask = senders != receivers
senders = senders[mask]
receivers = receivers[mask]

# Combine into edge_index
edge_index = np.stack([senders, receivers], axis=0)  # shape: [2, N*(N-1)]
print(edge_index.shape)

(2, 20)


# Save data in torch files .pt

In [ ]:
nodes_t_tensor = torch.tensor(nodes_t, dtype=torch.float32)
vel_updated_true_tensor = torch.tensor(vel_updated_true, dtype=torch.float32)
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)

torch.save(
    {
        'nodes': nodes_t_tensor,
        'velocities': vel_updated_true_tensor,
        'edge_index': edge_index_tensor
    },
    "simulation_data.pt"
)